# **공고문 수집기**
- 간단한 방법으로 작업들을 진행하기

In [1]:
import pandas as pd
table = pd.read_csv('data/busineSite.csv').iloc[:,:-1]
table = table.set_index('구분')
table['사이트'].to_dict()

{'나라장터': 'http://www.g2b.go.kr/index.jsp',
 '이나라도움': 'https://www.gosims.go.kr/hg/hg002/retrieveTaskSearchTop.do',
 '창업진흥원': 'https://www.k-startup.go.kr/main.do',
 '산업통상자원부': 'https://www.k-pass.kr/notice/ancList.do',
 '우리은행보조금시스템': 'https://ssd.wooribank.com/seoul/',
 '소상공인시장진흥공단': 'https://www.semas.or.kr/web/board/webBoardList.kmdc?bCd=1&pNm=BOA0101',
 '한국사회적기업진흥원': 'https://www.socialenterprise.or.kr/index.do#m_notice',
 '광진구청': 'https://www.gwangjin.go.kr/portal/bbs/B0000003/list.do?menuNo=200192',
 '서울50플러스 포털': 'https://www.50plus.or.kr/support.do',
 '서울시사회적경제지원센터': 'https://sehub.net/archives/category/alarm/opencat'}

In [2]:
from lxml.html import fromstring, tostring
import requests

def get_response(url):
    web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
    headers = {'User-Agent' : web_agent}
    response = requests.get(url, headers=headers, verify=False)
    resp_lxml = fromstring(response.text)
    return response.text, resp_lxml

# **서울시 사경센터 공고**
https://sehub.net/archives/category/alarm/opencat/page/1

In [3]:
url = 'https://sehub.net/archives/category/alarm/opencat/page/1'

# Url
import requests
web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
headers = {'User-Agent' : web_agent}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
response_text = response.text

from lxml.html import fromstring, tostring
resp_lxml = fromstring(response_text)
resp_table = resp_lxml.xpath("//table")[0]
table = pd.read_html(tostring(resp_table))[0]

link_urls = [_.get('href')  for _ in resp_table.xpath('.//a')]
table.insert(2,'link', link_urls)

index_filter = [no   for no, _ in enumerate(table['번호'].to_list())  if _ != '알림']
table = table.iloc[index_filter, :].reset_index(drop=True)
table.head(3)

,번호,제목,link,작성일,주최/주관
0,4588,[안내] 청년등 협동조합 창업지원사업 창업팀 모집 공고 (~5.6),https://sehub.net/archives/2065278,2021-04-16,HBM협동조합
1,4587,[안내] 사회적기업가 육성사업 환경분야 예비창업팀 모집 공고 (~5.28),https://sehub.net/archives/2065256,2021-04-16,한국사회적기업 진흥원 / 신나는조합
2,4586,[안내] 단비 대학생 아이디어 경진대회 개최 (~4.22),https://sehub.net/archives/2065239,2021-04-15,비전웍스


# **50플러스 공고**
- https://www.50plus.or.kr/support.do?page=2&
- lxml document links : http://i5on9i.blogspot.com/2017/03/lxml.html 

In [4]:
page = 1
url = f"https://www.50plus.or.kr/support.do?page={page}"

from requests_html import HTMLSession
session = HTMLSession()
response = session.get(url).text

from lxml.html import fromstring, tostring
resp_lxml = fromstring(response)
resp_texts = resp_lxml.xpath('//div[@class="board-box-wrapper"]')

# https://www.50plus.or.kr/detail.do?id=11554019
table_infos = []
for _ in resp_texts:
    info_items = _.xpath('.//p//text()')[:3]
    info_url = [_.get('href') for _ in _.xpath('.//a')][0]
    table_infos.append(info_items + ["https://www.50plus.or.kr/"+ info_url])

table = pd.DataFrame(table_infos)
table = table.iloc[:, [1,0,2,3]]
table.columns = ['일자','제목', '설명', 'link']
table.head(3)

,일자,제목,설명,link
0,2021-04-16,[서부][모집] 2021년 신중년커리어프로젝트 굿잡5060 참여자 모집,2021년 신중년커리어프로젝트 굿잡5060 참여자 모집 안내,https://www.50plus.or.kr/detail.do?id=10584887
1,2021-04-16,[남부]『50+다문화학습지원단』 최종합격자 발표 및 참여자 교육 안내,『50+다문화학습지원단』 최종합격자 발표 및 참여자 교육을 안내해드립니다.,https://www.50plus.or.kr/detail.do?id=11614619
2,2021-04-15,[노원]2021 서울50+인턴십 [노원50+지역사회브릿지 인턴십] 참여자 모집 공고,50+지역사회브릿지 인턴십 참여자 모집안내입니다.,https://www.50plus.or.kr/detail.do?id=11620587


# **e나라도움**
나라장터와 연관되어 사업내용을 확인하는 곳으로 주로 정산등의 작업에 활용 (겹치는 내용 여부는 확인해 볼 것)

- https://www.gosims.go.kr/hg/hg002/retrieveTaskReqstList.do
- https://www.gosims.go.kr/manual/ver2/4-9.%EC%A0%84%EC%B2%B4%EB%A9%94%EB%89%B4%EC%96%BC.pdf

In [5]:
mid=0"
bid	"701"
pageIndex	"1"

SyntaxError: EOL while scanning string literal (<ipython-input-5-b2de92bfdcef>, line 1)

In [ ]:
https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A&searchPrefixCode=BOARD_701_001&searchPostSn=131333&searchBusinessSn=583&

# **K-Start up**

- https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A&searchPrefixCode=BOARD_701_001&searchPostSn=131333&searchBusinessSn=583&

- https://www.k-startup.go.kr/common/announcement/announcementList.do?bid=701&searchAppAt=A
- https://www.k-startup.go.kr/common/announcement/announcementList.do?mid=30004&bid=701&searchAppAt=A
- https://www.k-startup.go.kr/common/announcement/announcementList.do?bid=701&searchAppAt=A&searchSortOrder=END_DATE   (마감일 정렬)
- https://www.k-startup.go.kr/common/announcement/announcementList.do?bid=701&searchAppAt=A&searchSortOrder=INSERT_DATE   (등록일 정렬)
```
<div class="ann_list_box">					
   <div class="list_total">전체건수<span>177</span>건</div>
   <div class="list_aling">
```

In [42]:
from lxml.html import fromstring
data = '<a id="hypProduct_10">link1</a>'
tree = fromstring(data)
tree.xpath('//a[re:test(@id, "^hypProduct_[0-9]+$")]', namespaces={'re': "http://exslt.org/regular-expressions"})[0].attrib["id"]

'hypProduct_10'

In [43]:
from lxml.html import tostring
url = "https://www.k-startup.go.kr/common/announcement/announcementList.do?bid=701&searchAppAt=A&searchSortOrder=INSERT_DATE"
resp, resp_lxml = get_response(url)
# resp

/home/momukji/Python/Torch/lib/python3.6/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.k-startup.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [44]:
resp_lxml.xpath('.//div[@class="ann_list_box"]')
items = resp_lxml.xpath('.//ul[@class="ann_list_impor"]/li')   # 주요안내 19개
items += resp_lxml.xpath('.//ul[@class="ann_list"]/li')        # 나머지 161개
len(items)

180

In [50]:
"".join(items[10].xpath('.//h4/span[1]//text()'))

'사업화'

In [49]:
"".join(items[10].xpath('.//a//text()')).strip()

'2021년 예비창업패키지 일반·특화 분야 예비창업자 모집 공고'

In [56]:
#  //a[contains(@class, "song")]
item_info = items[10].xpath('.//ul[contains(@class,"ann_list_info")]//li')
fo

[<Element li at 0x7f6250874d18>,
 <Element li at 0x7f6250874d68>,
 <Element li at 0x7f6250874c78>,
 <Element li at 0x7f6250874db8>]

In [ ]:
# ann_list_group01 

In [36]:
from lxml.html import fromstring
data = '<a id="hypProduct_10">link1</a>'
tree = fromstring(data)
tree.xpath('//a[re:test(@id, "^hypProduct_[0-9]+$")]', namespaces={'re': "http://exslt.org/regular-expressions"})[0].attrib["id"]

'hypProduct_10'

In [41]:
tostring(tree.xpath('//a[re:test(@id, "^hypProduct_[0-9]+$")]',namespaces={'re': "http://exslt.org/regular-expressions"})[0])

b'<a id="hypProduct_10">link1</a>'

In [19]:
items[0].xpath('.//h4//span//text()')

['사업화', '마감19일전', '\xa0']

In [20]:
items[18].xpath('.//text()')

['\r\n\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t',
 '사업화',
 '\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t',
 '마감11일전',
 '\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\t「IP기반 청년창업지원」 - 테크톤(Techthon) 모집공고\r\n\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t',
 '신청하기',
 '\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t',
 '\xa0',
 '\r\n\t\t\t\t\t\t\t\t\r\n\r\n\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t',
 'IP기반 청년창업지원',
 '\r\n\t\t\t\t\t\t\t\t',
 '차세대융합기술연구원',
 '\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t',
 '마감일자\xa0\xa02021-04-27',
 '\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\

In [ ]:
https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A&searchPrefixCode=BOARD_701_001&searchPostSn=131520

In [ ]:
https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A
    &searchPrefixCode=BOARD_701_001
    &searchPostSn=131520
    &searchBusinessSn=583&

In [21]:
# https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A
#     &searchPrefixCode=BOARD_701_001
#     &searchPostSn=131333
#     &searchBusinessSn=583&

import re
item_link = items[5].xpath('.//a')[0]
item_link = item_link.get('href')                # javascript Link 함수 찾기
item_link = re.findall(r'\(.*\)', item_link)[0]  # javascript 함수내 Params 찾기
item_link = item_link.replace('(','').replace(')','').replace("'", '')  # 불필요한 요소들 제거
item_link = tuple(map(str, item_link.split(', ')))
item_link_query = [_ for _ in item_link  if _ not in ['', '0', '_self']]
item_link, item_link_query

(('BOARD_701_001', '132713', '', '', '0', '671', '_self'),
 ['BOARD_701_001', '132713', '671'])

In [ ]:
# https://www.k-startup.go.kr/common/announcement/announcementDetail.do?bid=701&mid=0&searchAppAt=A
#     &searchPrefixCode=BOARD_701_001
#     &searchPostSn=131333
#     &searchBusinessSn=583&

import re
item_link = items[5].xpath('.//a')[0]
item_link = item_link.get('href')                # javascript Link 함수 찾기
item_link = re.findall(r'\(.*\)', item_link)[0]  # javascript 함수내 Params 찾기
item_link = item_link.replace('(','').replace(')','').replace("'", '')  # 불필요한 요소들 제거
item_link = tuple(map(str, item_link.split(', ')))
item_link_query = [_ for _ in res  if _ not in ['', '0', '_self']]
item_link_query

In [ ]:
item_title = items[4].xpath('.//text()')
"".join(item_title).strip()

In [ ]:
res[0]

In [ ]:
item_links

In [ ]:
searchPrefixCode=BOARD_701_001&searchPostSn=131333&searchBusinessSn=583&


In [ ]:
javascript:itemSelect('BOARD_701_001', '131333', '', '', '0', '583', '_self')

In [ ]:
searchPrefixCode	"BOARD_701_001"
searchPostSn	"131333"
searchBusinessSn	"583"

In [ ]:
items[0].xpath('.//h4//a//text()')

In [ ]:
items[0].xpath('.//text()')

# **산업통산자원부**
https://www.k-pass.kr/notice/ancList.do?pageIndex=1

In [ ]:
# Url
import requests
web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
headers = {'User-Agent' : web_agent}
response = requests.get(url, headers=headers)
response_text = response.text

from lxml.html import fromstring, tostring
resp_lxml = fromstring(response_text)
resp_table = resp_lxml.xpath("//table")[0]
table = pd.read_html(tostring(resp_table))[0]

link_urls = [_.get('href')  for _ in resp_table.xpath('.//a')]
table.insert(2,'link', link_urls)

index_filter = [no   for no, _ in enumerate(table['번호'].to_list())  if _ != '알림']
table = table.iloc[index_filter, :].reset_index(drop=True)
table.head(3)

In [ ]:
ancId=&gubun=&pageIndex=3&searchAncTitNm=&searchAncGrCd=&searchPrgWorkCd=&searchSrtDtm=&searchEndDtm=

In [ ]:
page = 1
url = f"https://www.50plus.or.kr/support.do?page={page}"

import requests
web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
headers   = {'User-Agent' : web_agent}
response  = requests.get(url, headers=headers).text

from lxml.html import fromstring, tostring
resp_lxml = fromstring(response_text)
resp_lxml.xpath('//div[contains(@class, "board-box-list")]')
# table = pd.read_html(tostring(resp_table))[0]

In [ ]:
with open('test50.html','w') as f:
    f.write(response)

In [ ]:
# ! pip install requests-html

In [ ]:
import requests
url = f"https://www.50plus.or.kr/support.do?page={page}"
_response = requests.get(url, headers=self.headers).text
resp_lxml  = fromstring(_response)


In [ ]:
from lxml.html import fromstring, tostring
import pandas as pd
import requests
import re


class GetHtml:
    
    def __init__(self):
        self.web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
        self.headers = {'User-Agent' : self.web_agent}
        self.html_meta = '<meta charset="'

    def info_seoul(self, page=1):
        r"""서울사경센터 공고 자동수집기"""
        # Web Page 의 Table 수집기
        url = f"https://sehub.net/archives/category/alarm/opencat/page/{page}"
        _response = requests.get(url, headers=self.headers).text
        resp_lxml  = fromstring(_response)
        resp_table = resp_lxml.xpath("//table")[0]
        table = pd.read_html(tostring(resp_table))[0]
        
        # Link href 수집기
        link_urls = [_.get('href')  for _ in resp_table.xpath('.//a')]

        # Post Procee : 테이블 합치기 및 공고내용 제거후 출력
        table.insert(2,'link', link_urls)
        index_filter = [no   for no, _ in enumerate(table['번호'].to_list())  if _ != '알림']
        return table.iloc[index_filter, :].reset_index(drop=True)
    
    def info_50plus(self, page=1):
        url = f"https://www.50plus.or.kr/support.do?page={page}"
        _response = requests.get(url, headers=self.headers).text
        resp_lxml  = fromstring(_response)


In [ ]:
url = "https://www.50plus.or.kr/support.do?page=2&"

import re, requests
def get_html(url):
    web_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2'
    headers = {'User-Agent' : web_agent}
    response = requests.get(url, headers=headers)
    html_meta = '<meta charset="'
    encode_name = "".join(re.findall(f'{html_meta}[A-z-0-9]+', str(response.content))).replace(html_meta,'')
    response.encoding = encode_name
    response_text = response.text
    return response_text


In [ ]:
get_html(url)

In [ ]:
url = "https://www.50plus.or.kr/support.do?page=2&"

In [ ]:
import requests
response = requests.get(url)
response

In [ ]:
import re

In [ ]:
re.findall(r'\<meta charset="[A-z-0-9]+', str(response.content))[0].replace('<meta charset="','')

In [ ]:
html_meta = '<meta charset="'
"".join(re.findall(f'{html_meta}[A-z-0-9]+', str(response.content))).replace(html_meta,'')

In [ ]:
join(['1','2'])

In [ ]:
text = response.content
text

In [ ]:
<meta charset="utf-8">

In [ ]:
response.content